In [ ]:
## ONLY FOR GOOGLE COLAB
!pip install tensorflow
!pip install keras
!pip install opencv-python

In [21]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, Conv2DTranspose
import os
import cv2 as cv
import numpy as np

In [22]:
conv_kwargs = {
    "padding"             : "SAME",
    "activation"          : keras.layers.LeakyReLU(alpha=0.2),
    "kernel_initializer"  : tf.random_normal_initializer(stddev=.1)
}

In [23]:
class Autoencoder(tf.keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.encoder = keras.Sequential([
             Conv2D(16, 8, 4, **conv_kwargs),
            Conv2D(16, 8, 4, **conv_kwargs),
            keras.layers.MaxPooling2D(),
            Conv2D(8, 4, 1, **conv_kwargs),
        ], name="ae_encoder")

        self.decoder = keras.Sequential([
        Conv2DTranspose(8, 4, 1, **conv_kwargs),
        Conv2DTranspose(16, 8, 4, **conv_kwargs),
        Conv2DTranspose(3, 8, 8, padding='same', kernel_initializer=tf.random_normal_initializer(stddev=.1))
    ], name='ae_decoder')

    def call(self, inputs):
        inputs = self.encoder(inputs)
        inputs = self.decoder(inputs)
        return inputs


In [24]:
def mse_bce_loss(y_true, y_pred):
    mse_loss = keras.losses.MeanSquaredError()     
    bce_loss = keras.losses.BinaryCrossentropy()
    mse = mse_loss(y_true, y_pred)
    bce = bce_loss(y_true, y_pred)
    loss = mse + bce
    return loss

In [25]:
ae_model = Autoencoder(name='autoencoder')

ae_model.build(input_shape = (1,256,256,3))   ## Required to see architecture summary
initial_weights = ae_model.get_weights() ## Just so we can reset out autoencoder

ae_model.summary()
ae_model.encoder.summary()
ae_model.decoder.summary()

ae_model.compile(
    optimizer   = keras.optimizers.legacy.Adam(learning_rate=0.001),
    loss        = mse_bce_loss,
    metrics     = [
        tf.keras.metrics.MeanSquaredError(),
        tf.keras.metrics.BinaryCrossentropy()
    ]
)

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ae_encoder (Sequential)     (1, 8, 8, 8)              21544     
                                                                 
 ae_decoder (Sequential)     (1, 256, 256, 3)          12315     
                                                                 
Total params: 33859 (132.26 KB)
Trainable params: 33859 (132.26 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "ae_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (1, 64, 64, 16)           3088      
                                                                 
 conv2d_10 (Conv2D)          (1, 16, 16, 16)           16400     
                                                               

In [26]:
import sys

isColab = "google.colab" in sys.modules
data_dir = '../collapsed_data'
# this also works:
# isColab = "COLAB_GPU" in os.environ

if isColab:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)

    data_dir = ("/content/drive/MyDrive/collapsed_data")

In [27]:
def get_data(sub_dir, size=(256,256)):
    dir = os.path.join(data_dir, sub_dir)
    files = os.listdir(dir)
    x = []
    y = []
    for f in files:
        path = os.path.join(dir, f)
        try:
            img = cv.imread(path)
            img = cv.resize(img, size)
            y.append(img)
        except:
            print(path)
            continue
    y = np.array(y, dtype=np.float32)
    y = y[:4000]
    x = np.copy(y)
    h,w = size
    rec_w = w//5
    # Make middle black
    x[:,:,rec_w*2:rec_w*3,:]=0
    y=y/255
    x = x/255
    # cv.imshow('x',x[21])
    # cv.waitKeyEx()
    # cv.imshow('y',y[21])
    # cv.waitKeyEx()
    print(x.shape)
    print(y.shape)
    print('done')
    return x,y


In [2]:
x_train,y_train = get_data('train')
#x_valid,y_valid = get_data('validation')

NameError: name 'get_data' is not defined

In [ ]:
# Train the model
print('Fitting model')
#ae_model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_valid,y_valid))
ae_model.fit(x_train, y_train, epochs=5, batch_size=64)
print('------------------------')

In [ ]:
ae_model.save('models/model.keras')

In [28]:
ae_model = keras.models.load_model('model.keras',custom_objects={'Autoencoder': Autoencoder,'mse_bce_loss':mse_bce_loss})

In [29]:
print('Evaluating model on testing data')
x_test,y_test = get_data('test')
ae_model.evaluate(x_test, y_test, batch_size=32)

Evaluating model on testing data
(500, 256, 256, 3)
(500, 256, 256, 3)
done
16/16 [==============================] - 1s 28ms/step - loss: 0.6212 - mean_squared_error: 0.0308 - binary_crossentropy: 0.5904


[0.6211981177330017, 0.03077639825642109, 0.5904217958450317]

In [30]:
x = x_test[:5]
pred = ae_model.predict(x)
for i in range(5):
    cv.imshow('im', x[i])
    cv.waitKeyEx()
    cv.imshow('pred', pred[i])
    cv.waitKeyEx()

1/1 [==============================] - 0s 79ms/step
